In [1]:
import os
import sqlite3
from sqlite3 import Error

In [2]:
def create_connection(path):
    connection=None
    try:
        connection=sqlite3.connect(path)
        print("connection to sqliteDB successful")
    except Error as e:
        print(f"The error '{e}' occurred")
        
    return connection

In [30]:
def execute_query(query,connection=connection):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

In [35]:
def execute_read_query(query, connection=connection):
    cursor=connection.cursor()
    result=None
    try:
        cursor.execute(query)
        result=cursor.fetchall()
        return result
    except Error as e:
        print("The error '{e}' occurred")

In [8]:
working_dir = "C:\\Users\\saman\\Documents\\Python"
db_obj = "test.sqlite"

In [9]:
connection=create_connection(working_dir+db_obj)

connection to sqliteDB successful


In [17]:
# the query to create a table with the following columns: id, name, age, gender, nationality

create_users_table="""
CREATE TABLE IF NOT EXISTS users (
id INTEGER PRIMARY KEY AUTOINCREMENT,
name TEXT NOT NULL,
age INTEGER,
gender TEXT,
nationality TEXT
);
"""

In [18]:
execute_query(create_users_table)

Query executed successfully


In [22]:
# the query creates the posts table
create_posts_table="""
CREATE TABLE IF NOT EXISTS posts(
id INTEGER PRIMARY KEY AUTOINCREMENT,
title TEXT NOT NULL,
description TEXT NOT NULL,
user_id INTEGER NOT NULL,
FOREIGN KEY (user_id) REFERENCES users (id)
);
"""

In [23]:
execute_query(create_posts_table)

Query executed successfully


In [24]:
create_comments_table="""
CREATE TABLE IF NOT EXISTS comments(
id INTEGER PRIMARY KEY AUTOINCREMENT,
text TEXT NOT NULL,
user_id INTEGER NOT NULL,
post_id INTEGER NOT NULL,
FOREIGN KEY (user_id) REFERENCES user (id) FOREIGN KEY (post_id) REFERENCES posts (id)
);
"""

create_likes_table="""
CREATE TABLE IF NOT EXISTS likes(
id INTEGER PRIMARY KEY AUTOINCREMENT,
user_id INTEGER NOT NULL,
post_id INTEGER NOT NULL,
FOREIGN KEY (user_id) REFERENCES users (id) FOREIGN KEY (post_id) REFERENCES posts (id)
);
"""

In [32]:
execute_query(create_comments_table)
execute_query(create_likes_table)

Query executed successfully
Query executed successfully


In [33]:
create_users="""
INSERT INTO
    users(name,age,gender,nationality)
    
VALUES
    ('James',25,'male','USA'),
    ('Leila',32,'female','France'),
    ('Brigitte',35,'female','UK'),
    ('Mike',40,'male','Denmark'),
    ('Elizabeth',21,'female','Canada');
"""

In [34]:
execute_query(create_users)

Query executed successfully


In [36]:
execute_read_query("SELECT * FROM users")

[(1, 'James', 25, 'male', 'USA'),
 (2, 'Leila', 32, 'female', 'France'),
 (3, 'Brigitte', 35, 'female', 'UK'),
 (4, 'Mike', 40, 'male', 'Denmark'),
 (5, 'Elizabeth', 21, 'female', 'Canada')]

In [37]:
create_posts="""
INSERT INTO
    posts(title,description,user_id)
VALUES
    ("Happy", "I am feeling very happy today",1),
    ("Hot weather", "The weather is very hot today", 2),
    ("Help", "I need some help with my work",3),
    ("Getting it", "I think this is becoming clearer",4),
    ("Interesting Game","It was a great game of tennis", 5),
    ("Party","Anyone up for a party", 3);
"""
# The user_id column of posts is a FOREIGN KEY and refers to a key in another table as set during creation, in this case
# the user_id column must contain a value that already exists in the id column of users as this is the one we refer it to.

# If this is not the case, an error will occur.

In [39]:
execute_query(create_posts)

Query executed successfully


In [41]:
execute_read_query("SELECT * FROM posts")

[(1, 'Happy', 'I am feeling very happy today', 1),
 (2, 'Hot weather', 'The weather is very hot today', 2),
 (3, 'Help', 'I need some help with my work', 3),
 (4, 'Getting it', 'I think this is becoming clearer', 4),
 (5, 'Interesting Game', 'It was a great game of tennis', 5),
 (6, 'Party', 'Anyone up for a party', 3)]

In [43]:
create_comments="""
INSERT INTO
    comments(text,user_id, post_id)
VALUES
    ('Count me in', 1, 6),
    ('What sort of help', 5, 3),
    ('Congrats', 4, 4),
    ('I was rooting for the other player', 4, 5),
    ('Help with your thesis?', 2, 3);
"""

create_likes="""
INSERT INTO
    likes(user_id,post_id)
VALUES
    (1,6),
    (2,3),
    (1,5),
    (5,4),
    (2,4),
    (4,2),
    (3,6);

"""

In [44]:
execute_query(create_comments)
execute_query(create_likes)

Query executed successfully
Query executed successfully


In [ ]:
# implementation of JOIN operations

In [49]:
select_user_posts="""
SELECT
    users.id,
    users.name,
    posts.description
FROM
    posts
    INNER JOIN  users ON users.id = posts.user_id
"""

In [50]:
execute_read_query(select_user_posts)

[(1, 'James', 'I am feeling very happy today'),
 (2, 'Leila', 'The weather is very hot today'),
 (3, 'Brigitte', 'I need some help with my work'),
 (4, 'Mike', 'I think this is becoming clearer'),
 (5, 'Elizabeth', 'It was a great game of tennis'),
 (3, 'Brigitte', 'Anyone up for a party')]

In [52]:
# multiple join functions

select_posts_comments_users="""
SELECT
    posts.description as post,
    text as comment,
    name
FROM
    posts
    INNER JOIN comments ON posts.id = comments.post_id
    INNER JOIN users ON users.id = comments.user_id
"""

In [53]:
execute_read_query(select_posts_comments_users)

[('Anyone up for a party', 'Count me in', 'James'),
 ('I need some help with my work', 'What sort of help', 'Elizabeth'),
 ('I think this is becoming clearer', 'Congrats', 'Mike'),
 ('It was a great game of tennis',
  'I was rooting for the other player',
  'Mike'),
 ('I need some help with my work', 'Help with your thesis?', 'Leila')]

In [65]:
# The column names are not being returned by the .fetchall() function. To return column names you can use description
# attribute of the cursor

In [68]:
cursor=connection.cursor()
cursor.execute(select_posts_comments_users)
cursor.fetchall()

column_names = [description[0] for description in cursor.description]
print(column_names)

['post', 'comment', 'name']


In [ ]:
# implementation of WHERE operations

In [57]:
select_posts_likes="""
SELECT
    description as Post,
    COUNT(likes.id) as Likes
FROM
    likes,
    posts
WHERE
    posts.id=likes.post_id
GROUP BY
    likes.post_id
"""

In [58]:
execute_read_query(select_posts_likes)

[('The weather is very hot today', 1),
 ('I need some help with my work', 1),
 ('I think this is becoming clearer', 2),
 ('It was a great game of tennis', 1),
 ('Anyone up for a party', 2)]

In [60]:
# Updating records using the execute_query function

In [61]:
execute_read_query("SELECT description FROM posts WHERE id=2")

[('The weather is very hot today',)]

In [62]:
update_post_description="""
UPDATE
    posts
SET
    description="The weather has become pleasant now"
WHERE
    id=2
"""

In [63]:
execute_query(update_post_description)

Query executed successfully


In [64]:
execute_read_query("SELECT description FROM posts WHERE id=2")

[('The weather has become pleasant now',)]

In [ ]:
# Deleting tables records

In [78]:
delete_comment = "DELETE FROM comments WHERE id=7" # There is no id 7, so nothing will be removed from the test data.
execute_query(delete_comment)

Query executed successfully
